# Langchain and Anthropic

Claude is chat-based model, meaning it is trained on conversation data. However, it is a text based API, meaning it takes in single string. It expects this string to be in a particular format. This means that it is up the user to ensure that is the case. LangChain provides several utilities and helper functions to make sure prompts that you write - whether formatted as a string or as a list of messages - end up formatted correctly.

Because Claude is chat-based but accepts a string as input, it can be treated as either a LangChain `ChatModel` or `LLM`. This means there are two wrappers in LangChain - ChatAnthropic and Anthropic. It is generally recommended to use the ChatAnthropic wrapper, and format your prompts as ChatMessages (we will show examples of this below). This is because it keeps your prompt in a general format that you can easily then also use with other models (should you want to). However, if you want more fine-grained control over the prompt, you can use the Anthropic wrapper - we will show and example of this as well. The `Anthropic` wrapper however is **deprecated**, as all functionality can be achieved in a more generic way using `ChatAnthropic`.

Ref: <https://python.langchain.com/docs/integrations/platforms/anthropic>

## Best Practices

#### No System Messages

Anthropic models are not trained on the concept of a "system message". We have worked with the Anthropic team to handle them somewhat appropriately (a Human message with an admin tag) but this is largely a hack and it is recommended that you do not use system messages.

#### AI Messages Can Continue

A completion from Claude is a continuation of the last text in the string which allows you further control over Claude's output.

For example, putting words in Claude's mouth in a prompt like this:

>`\n\nHuman: Tell me a joke about bears\n\nAssistant: What do you call a bear with no teeth?`

This will return a completion like this `A gummy bear!` instead of a whole new assistant message with a different random bear joke.

In [ ]:
# !pip install -Uq boto3 anthropic-bedrock langchain rich

In [ ]:
%load_ext rich

In [ ]:
from langchain.chat_models import ChatAnthropic
from langchain.llms.bedrock import Bedrock
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.llm import LLMChain
from rich import print
import boto3
from anthropic_bedrock import HUMAN_PROMPT, AI_PROMPT
from utils import get_inference_parameters
import warnings
warnings.filterwarnings("ignore")

### Initialize Bedrock LLM

Here we utilize `langchain.llms.bedrock.Bedrock` class to initialize our LLM.

In [ ]:
# get bedrock runtime client
client = boto3.client("bedrock-runtime", region_name="us-west-2")

bedrock_model_id = "anthropic.claude-v2"  # Bedrock model_id
model_kwargs = get_inference_parameters("anthropic")  # Model kwargs for Anthropic LLMs
# print(model_kwargs)

bedrock_model = Bedrock(
    client=client, model_id=bedrock_model_id, model_kwargs=model_kwargs
)  # Initalize LLM

#### Creating prompts with ChatPromptTemplate

When working with ChatModels, it is preferred that you design your prompts as `ChatPromptTemplates`.

Here is an example below of doing that:

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful chatbot"),
    ("human", "Hi! How are you?"),
    ("assistant", "I'm doing well. Thanks for asking."),
    ("human", "Tell me a joke about {topic}"),
])
print(prompt)

You can then invoke the chain like below:

*NOTE:* The below example is using Langchain Expression Language (LCEL)

Refer to <https://python.langchain.com/docs/expression_language> for more details.

In [ ]:
chain = prompt | bedrock_model
chain.invoke({"topic": "cats"})

### Creating Prompts with ChatPromptTemplate

Below we use `HumanMessage` from `langchain.schema` to construct `messages` for the chat model.

We then create a `ChatPromptTemplate` from these messages to invoke this chain.


*NOTE: As there are no input variables to be passed, we send an empty `input` string*

In [ ]:
from langchain.schema import HumanMessage

messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
prompt = ChatPromptTemplate.from_messages(messages=messages)

chain = prompt | bedrock_model
chain.invoke({"input": ""})

### Creating Prompts with ChatPromptTemplate

We can see that under the hood LangChain is not appending any prefix/suffix to SystemMessage's. This is because `Anthropic` has no concept of `SystemMessage`. 

Anthropic requires all prompts to end with assistant messages. This means if the last message is not an assistant message, the suffix `Assistant:` will automatically be inserted.

If you decide instead to use a normal `PromptTemplate` (one that just works on a single string) then we need do the following:

- Prefix our prompt string with `HUMAN_PROMPT` and suffix with `AI_PROMPT`

In [ ]:
from anthropic_bedrock import HUMAN_PROMPT, AI_PROMPT
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
# Format prompt for Anthropic
anthropic_prompt = PromptTemplate.from_template(f"{HUMAN_PROMPT}{prompt.template}{AI_PROMPT}")
print(anthropic_prompt)

# Invoke the Chain
chain = anthropic_prompt | bedrock_model
chain.invoke({"topic": "otters"})

## Sequential Chain

1. Simple Sequential Chain
2. Complex Sequential Chain

### Simple Sequential Chain

**CHAIN_1** -> [*OUTPUT*] -> **CHAIN_2**

In [ ]:
from langchain.chains import SimpleSequentialChain
from langchain.chat_models import ChatAnthropic

model = ChatAnthropic(anthropic_api_key="q2342a5-34534544")

llm = bedrock_model

# prompt template 1
first_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "What is the best name to describe a company that makes {product}? output just the best name you can think of. Only output one name and nothing else.",
        ),
    ]
)

# Chain 1
s_chain_one = LLMChain(llm=llm, prompt=first_prompt)
# print(s_chain_one)

In [ ]:
# prompt template 1
second_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "Write a 20 words description for the following company: {company_name}",
        ),
    ]
)

# Chain 1
s_chain_two = LLMChain(llm=llm, prompt=second_prompt)
# print(s_chain_two)

In [ ]:
overall_simple_chain = SimpleSequentialChain(
    chains=[s_chain_one, s_chain_two], verbose=True
)

# print(overall_simple_chain)

#### Run simple sequential chain

In [ ]:
product = "Queen Size Sheet Set"
overall_simple_chain.run(product)

### Complex Sequential Chain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
# input = "La nourriture était médiocre. J'ai aimé le steak et la sauce qui l'accompagne. A part ça, tout le reste n'était que moi."
input = "Das Essen war mittelmäßig. Mir gefielen das Steak und die dazugehörige Soße. Ansonsten war alles andere nur mittelmäßig."

In [ ]:
first_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Translate the following review to english:" "\n\n{review}"),
    ]
)

chain_one = LLMChain(llm=bedrock_model, prompt=first_prompt, output_key="eng_review")

# print(chain_one)

In [ ]:
second_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "Can you summarize the following review in 1 sentence:" "\n\n{eng_review}",
        ),
    ]
)

# chain 2: input= eng_review and output= summary
chain_two = LLMChain(llm=bedrock_model, prompt=second_prompt, output_key="summary")
# print(chain_two)

In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What language is the following review:\n\n{eng_review}"),
    ]
)

# chain 3: input= Review and output= language
chain_three = LLMChain(llm=bedrock_model, prompt=third_prompt, output_key="language")
# print(chain_three)

In [ ]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "Write a follow up response to the following "
            "summary in the specified language:"
            "\n\nSummary: {summary}\n\nLanguage: {language}",
        ),
    ]
)

# chain 3: input= Review and output= language
chain_four = LLMChain(
    llm=bedrock_model, prompt=fourth_prompt, output_key="followup_message"
)
# print(chain_four)

In [ ]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["eng_review", "summary", "followup_message"],
    verbose=True,
)
# print(overall_chain)

In [ ]:
input

In [ ]:
overall_chain(input)

In [ ]:
# from anthropic_bedrock import HUMAN_PROMPT, AI_PROMPT
# prompt_file = Path("./function_calling_prompt.txt")
# prompt = PromptTemplate.from_file(prompt_file, ["tools_string", "user_input"])

# # Prefix prompt with HUMAN_PROMPT and suffix with AI_PROMPT
# prompt.template = f"{HUMAN_PROMPT}{prompt.template}{AI_PROMPT}"
# print(prompt)

In [ ]:
# def output_parser(text):
#     """
#     Function to extract text within <answer> tags

#     input: text (str) - output from LLM

#     output: extracted_text (str) - returns extracted text from <answer> tags if found.
#     Else, returns "no answer tags found"

#     """
#     match = re.search(r"<answer>(.*?)</answer>", text, re.DOTALL)
#     if match:
#         extracted_text = match.group(1)
#     else:
#         extracted_text = "no answer tags found"
#     return extracted_text

# def extract_func_calls(llm_output: str):
#     pattern = r"<function_call>(.*?)</function_call>"
#     matches = re.findall(pattern, llm_output, re.DOTALL)
#     return matches[0]
#     # for match in matches:
#     #     print(match)

# # Prepare input variables
# tools_string = add_tools()
# question = "What is the weather in Denver"

# chain_input = {
#     "tools_string": tools_string,
#     "user_input" : question
# }

# # create chain using LCEL
# chain = prompt | bedrock_model | extract_func_calls | execute_function

# # Invoke chain
# print(f"Invoking func calling chain to answer: [b]{question}[/b]")

# output = chain.invoke(chain_input)
# print(output)